In [190]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option("max_colwidth", None)

In [191]:
# load anime dataset
anime_df = pd.read_csv('dataset/anime.csv')
usecols = ["MAL_ID", "Name", "Score", "Genres", "Type", "Episodes", "Premiered",
           "Studios", "Source", "Rating", "Completed"]

anime_df=pd.read_csv('dataset/anime.csv',usecols=usecols)
anime_df.head()

,MAL_ID,Name,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Completed
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Spring 1998,Sunrise,Original,R - 17+ (violence & profanity),718161
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Unknown,Bones,Original,R - 17+ (violence & profanity),208333
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Spring 1998,Madhouse,Manga,PG-13 - Teens 13 or older,343492
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, Magic",TV,26,Summer 2002,Sunrise,Original,PG-13 - Teens 13 or older,46165
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",TV,52,Fall 2004,Toei Animation,Manga,PG - Children,7314


In [192]:
# load top users with anime rating dataset
top_animes_count = 1000
rating_popular_anime = pd.read_csv('dataset/top_anime_unsupervised_use.csv')
animes_popular = anime_df.sort_values('Completed', ascending=False)[0:top_animes_count]
top_animes = animes_popular['MAL_ID'].unique()
rating_popular_anime = rating_popular_anime[rating_popular_anime.MAL_ID.isin(top_animes)]
rating_popular_anime.head()

,user_id,MAL_ID,rating
3,478,20,7
5,478,24,8
7,478,43,6
8,478,47,8
9,478,48,6


In [193]:
# user item interaction matrix
anime_matrix_UII=rating_popular_anime.pivot_table(index='user_id',columns='MAL_ID',values='rating').fillna(0)
anime_matrix_UII.head()

MAL_ID,1,5,6,18,19,20,24,30,31,32,...,40716,40776,40839,40902,40956,41120,41168,41226,41353,41389
user_id,,,,,,,,,,,,,,,,,,,,,
478,0.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
781,7.0,0.0,10.0,8.0,10.0,7.0,7.0,0.0,0.0,0.0,...,8.0,9.0,7.0,4.0,7.0,0.0,6.0,7.0,7.0,3.0
853,9.0,10.0,8.0,9.0,8.0,9.0,0.0,8.0,0.0,8.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
890,9.0,9.0,9.0,8.0,10.0,7.0,2.0,8.0,7.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
912,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,8.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0


In [201]:
test_user_rating = anime_matrix_UII[1]
test_user_rating

user_id
478        0.0
781        7.0
853        9.0
890        9.0
912        0.0
          ... 
352832     0.0
352835     0.0
352922     0.0
352924    10.0
352930    10.0
Name: 1, Length: 2998, dtype: float64

In [195]:
# generalize above exploration to a fit function
def predict(input_MAL_ID, recommend_count):
    '''
    :return: dataframe of recommended anime
    '''
    user_rating_input = anime_matrix_UII[input_MAL_ID]
    similar_to_input_anime = anime_matrix_UII.corrwith(user_rating_input)
    corr_input = pd.DataFrame(similar_to_input_anime, columns=['correlation'])
    corr_input.dropna(inplace=True)
    corr_input.reset_index(inplace=True)
    corr_input = pd.merge(corr_input, anime_df, on="MAL_ID")
    recommendation = corr_input[corr_input['Completed'] > 10000].sort_values(by='correlation', ascending=False).head(recommend_count+1)
    recommendation.drop(recommendation.index[recommendation['MAL_ID'] == input_MAL_ID], inplace=True)
    return recommendation

In [196]:
# test functionality on another anime
recommendation_test = predict(1, 5)
recommendation_test

,MAL_ID,correlation,Name,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Completed
1,5,0.653634,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Unknown,Bones,Original,R - 17+ (violence & profanity),208333
2,6,0.468294,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Spring 1998,Madhouse,Manga,PG-13 - Teens 13 or older,343492
49,205,0.440870,Samurai Champloo,8.5,"Action, Adventure, Comedy, Historical, Samurai, Shounen",TV,26,Spring 2004,Manglobe,Original,R - 17+ (violence & profanity),551621
84,467,0.420354,Koukaku Kidoutai: Stand Alone Complex,8.45,"Action, Military, Sci-Fi, Police, Mecha, Seinen",TV,26,Fall 2002,Production I.G,Manga,R - 17+ (violence & profanity),170891
11,43,0.397354,Koukaku Kidoutai,8.29,"Action, Mecha, Police, Psychological, Sci-Fi, Seinen",Movie,1,Unknown,Production I.G,Manga,R+ - Mild Nudity,325682


In [197]:
recommendation_for_all = animes_popular[:100].sort_values(by='Score', ascending=False)[:10]
top_users = rating_popular_anime.user_id.unique()

In [204]:
def recommend_for_user(user_id_input):
    '''
    Take user_id to get top-rated anime by the user, then for each anime make prediction
    return the highest 10 correlation anime
    :param user_id_input: user id
    :return: dataframe of recommended anime
    '''
    recommendation_df = pd.DataFrame(columns=['MAL_ID', 'correlation'])
    users_groups = rating_popular_anime.groupby('user_id')
    if user_id_input in top_users:
        user_fav_anime = users_groups.get_group(user_id_input).sort_values(by='rating', ascending=False)[:5]['MAL_ID'].tolist()
        for user_fav_MAL_ID in user_fav_anime:
            r = predict(user_fav_MAL_ID, 10)
            recommendation_df = recommendation_df.append(r, ignore_index=True)

        recommendation_df.sort_values('correlation', ascending=False).drop_duplicates('MAL_ID')
        return recommendation_df[:10]
    else:
        return recommendation_for_all

In [207]:
recommend_for_user(2162)

,MAL_ID,correlation,Name,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Completed
0,467,0.851933,Koukaku Kidoutai: Stand Alone Complex,8.45,"Action, Military, Sci-Fi, Police, Mecha, Seinen",TV,26,Fall 2002,Production I.G,Manga,R - 17+ (violence & profanity),170891.0
1,468,0.622324,Innocence,7.82,"Military, Sci-Fi, Police, Psychological, Mecha",Movie,1,Unknown,Production I.G,Manga,R - 17+ (violence & profanity),98216.0
2,43,0.569361,Koukaku Kidoutai,8.29,"Action, Mecha, Police, Psychological, Sci-Fi, Seinen",Movie,1,Unknown,Production I.G,Manga,R+ - Mild Nudity,325682.0
3,790,0.395838,Ergo Proxy,7.92,"Psychological, Mystery, Sci-Fi",TV,23,Winter 2006,Manglobe,Original,R - 17+ (violence & profanity),213453.0
4,5,0.392451,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Unknown,Bones,Original,R - 17+ (violence & profanity),208333.0
5,1,0.374166,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Spring 1998,Sunrise,Original,R - 17+ (violence & profanity),718161.0
6,1519,0.346066,Black Lagoon: The Second Barrage,8.19,"Action, Seinen",TV,12,Fall 2006,Madhouse,Manga,R - 17+ (violence & profanity),303731.0
7,889,0.331493,Black Lagoon,8.05,"Action, Seinen",TV,12,Spring 2006,Madhouse,Manga,R - 17+ (violence & profanity),435287.0
8,323,0.326610,Mousou Dairinin,7.68,"Mystery, Dementia, Police, Psychological, Supernatural, Drama, Thriller",TV,13,Winter 2004,Madhouse,Original,R+ - Mild Nudity,136786.0
9,6,0.316695,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Spring 1998,Madhouse,Manga,PG-13 - Teens 13 or older,343492.0
